# Oscillatory Shear Test - April 14th 2022

In [ ]:
import numpy as np

import matplotlib.pyplot as plt

from monk import nb

from typing import Callable, Optional

import hoomd
import gsd.hoomd

from monk import prep
from monk import pair

import ex_render

In [ ]:
seed = 124
device = hoomd.device.auto_select()
sim = hoomd.Simulation(device, seed=seed)

N = 512
rng = prep.init_rng(seed + 1)
snap = prep.approx_euclidean_snapshot(N, 25, rng, ratios=[50,50], diams=[1.0, 1.4])

sim.create_state_from_snapshot(snap)
sim.state.thermalize_particle_momenta(hoomd.filter.All(), 0.01)
# sim.always_compute_pressure = True

In [ ]:
ex_render.render_disk_frame(frame=sim.state.get_snapshot())

In [ ]:
integrator = hoomd.md.Integrator(dt=0.001)

nlist = hoomd.md.nlist.Tree(0.3)
hertz = pair.bi_hertz(nlist)
nve = hoomd.md.methods.NVE(hoomd.filter.All())
integrator.forces = [hertz]
integrator.methods = [nve]

sim.operations.integrator = integrator

In [ ]:
sim.run(0)

In [ ]:
sim.run(10_000)

In [ ]:
sim.state.get_snapshot().particles.position

In [ ]:
ex_render.render_disk_frame(frame=sim.state.get_snapshot())

In [ ]:
variant = hoomd.variant.Ramp(0, 1, sim.timestep, t_ramp=10_000)

initial_box = sim.state.box
final_box = hoomd.Box.from_box(initial_box)  # make a copy of initial_box
final_rho = 1.4
final_box.volume = sim.state.N_particles / final_rho

resize_box = hoomd.update.BoxResize(hoomd.trigger.Periodic(10), sim.state.box, final_box, variant)

sim.operations.updaters.clear()
sim.operations.updaters.append(resize_box)

In [ ]:
sim.state.box

In [ ]:
sim.run(10_000)

In [ ]:
ex_render.render_disk_frame(frame=sim.state.get_snapshot())

In [ ]:
integrator.forces.pop()

In [ ]:
fire = hoomd.md.minimize.FIRE(dt=0.05,
                        force_tol=1e-2,
                        angmom_tol=1e-2,
                        energy_tol=1e-7)
fire.methods.append(hoomd.md.methods.NVE(hoomd.filter.All()))
fire.forces = [hertz]
sim.operations.integrator = fire

In [ ]:
sim.run(1_000)

In [ ]:
ex_render.render_disk_frame(frame=sim.state.get_snapshot())